## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kamaishi,JP,39.2667,141.8833,31.15,83,100,6.62,light snow
1,1,Marsala,IT,37.7966,12.4352,51.35,62,0,6.42,clear sky
2,2,Wiarton,CA,44.7407,-81.1332,37.04,75,100,11.50,overcast clouds
3,3,Saint George,US,37.1041,-113.5841,59.59,29,0,25.32,clear sky
4,4,Namatanai,PG,-3.6667,152.4333,84.31,72,66,6.62,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Namatanai,PG,-3.6667,152.4333,84.31,72,66,6.62,light rain
9,9,Hithadhoo,MV,-0.6000,73.0833,82.00,73,42,11.01,scattered clouds
12,12,Bathsheba,BB,13.2167,-59.5167,75.81,73,75,12.66,broken clouds
26,26,Kapaa,US,22.0752,-159.3190,76.98,74,38,6.22,scattered clouds
42,42,Georgetown,MY,5.4112,100.3354,84.13,83,40,6.91,scattered clouds
46,46,Moerewa,NZ,-35.3833,174.0333,78.35,70,99,6.87,overcast clouds
51,51,Ahipara,NZ,-35.1667,173.1667,78.75,70,100,10.69,overcast clouds
55,55,Port Hueneme,US,34.1478,-119.1951,76.57,30,36,6.20,scattered clouds
56,56,Alice Springs,AU,-23.7000,133.8833,89.15,27,16,10.36,few clouds
65,65,Kupang,ID,-10.1667,123.5833,82.60,80,95,6.78,light rain


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             150
City                   150
Country                150
Lat                    150
Lng                    150
Max Temp               150
Humidity               150
Cloudiness             150
Wind Speed             150
Current Description    150
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#NA

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Namatanai,PG,84.31,light rain,-3.6667,152.4333,
9,Hithadhoo,MV,82.00,scattered clouds,-0.6000,73.0833,
12,Bathsheba,BB,75.81,broken clouds,13.2167,-59.5167,
26,Kapaa,US,76.98,scattered clouds,22.0752,-159.3190,
42,Georgetown,MY,84.13,scattered clouds,5.4112,100.3354,
46,Moerewa,NZ,78.35,overcast clouds,-35.3833,174.0333,
51,Ahipara,NZ,78.75,overcast clouds,-35.1667,173.1667,
55,Port Hueneme,US,76.57,scattered clouds,34.1478,-119.1951,
56,Alice Springs,AU,89.15,few clouds,-23.7000,133.8833,
65,Kupang,ID,82.60,light rain,-10.1667,123.5833,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
      
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Namatanai,PG,84.31,light rain,-3.6667,152.4333,Seagulls Inn Namatani
9,Hithadhoo,MV,82.00,scattered clouds,-0.6000,73.0833,Scoop Guest House
12,Bathsheba,BB,75.81,broken clouds,13.2167,-59.5167,Atlantis Hotel
26,Kapaa,US,76.98,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
42,Georgetown,MY,84.13,scattered clouds,5.4112,100.3354,Cititel Penang
46,Moerewa,NZ,78.35,overcast clouds,-35.3833,174.0333,Hunter Star Hotel
51,Ahipara,NZ,78.75,overcast clouds,-35.1667,173.1667,Ahipara Holiday Park
55,Port Hueneme,US,76.57,scattered clouds,34.1478,-119.1951,Hampton Inn Channel Islands Harbor
56,Alice Springs,AU,89.15,few clouds,-23.7000,133.8833,Desert Palms Alice Springs
65,Kupang,ID,82.60,light rain,-10.1667,123.5833,Hotel On The Rock by Prasanthi


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))